#### Installesion

In [3]:
import h2o
h2o.init()
h2o.demo("glm")

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,17 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_sysadm_e4ekox
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.906 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


-------------------------------------------------------------------------------
Demo of H2O's Generalized Linear Estimator.

This demo uploads a dataset to h2o, parses it, and shows a description.
Then it divides the dataset into training and test sets, builds a GLM
from the training set, and makes predictions for the test set.
Finally, default performance metrics are displayed.
-------------------------------------------------------------------------------

>>> # Connect to H2O
>>> h2o.init()

(press any key)

error: (25, 'Inappropriate ioctl for device')

In [4]:
#create cluster
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /home/sysadm/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpabnskykk
  JVM stdout: /tmp/tmpabnskykk/h2o_sysadm_started_from_python.out
  JVM stderr: /tmp/tmpabnskykk/h2o_sysadm_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_sysadm_d5ai4l
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.906 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [6]:
#import a dataset

loans = h2o.import_file("https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-at-Scale-with-H2O/main/chapt2/loans-lite.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
loans.describe()

Rows:5000
Cols:16

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,credit_length
type,int,enum,real,int,enum,real,enum,int,enum,enum,real,int,real,int,int,int
mins,500.0,,5.42,0.0,,3300.0,,1180656000000.0,,,0.0,0.0,0.0,1.0,0.0,0.0
mean,10757.399999999998,,12.042216000000003,4.740286298568507,,68984.26325465094,,1284307868160.0,,,13.212278,0.14031224979983986,49.15658321650311,21.830064051240992,0.155,13.413130504403522
maxs,35000.0,,23.91,10.0,,3900000.0,,1322697600000.0,,,29.93,6.0,106.5,73.0,1.0,52.0
sigma,7173.197679068143,,3.612830457982719,3.580079530043472,,78076.24567859182,,32089484313.69493,,,6.71513728560101,0.4702652144800979,28.367928141780574,11.634565225391304,0.361940879481785,6.844231383509271
zeros,0,,0,659,,0,,0,,,26,4470,124,0,4225,1
missing,0,0,0,110,0,1,0,0,0,0,0,4,7,4,0,4
0,5000.0,36 months,11.97,1.0,OWN,51320.0,not verified,1207008000000.0,debt_consolidation,CA,22.73,0.0,70.3,11.0,0.0,10.0
1,18000.0,60 months,13.49,10.0,RENT,72000.0,verified,1317427200000.0,debt_consolidation,PA,19.1,0.0,74.8,13.0,0.0,16.0
2,11975.0,60 months,15.21,3.0,MORTGAGE,64584.0,verified,1285891200000.0,debt_consolidation,MD,19.81,0.0,61.1,17.0,0.0,8.0


In [9]:
#split

train,validation = loans.split_frame(ratios=[.75])
label= "bad_loan"
predictors = loans.col_names
predictors.remove(label)

In [10]:
#build the model

from h2o.estimators import H2OXGBoostEstimator
param = {"ntrees" : 25, "nfolds" : 10}
xgboost_model = H2OXGBoostEstimator(**param)
xgboost_model.train(x = predictors,
                    y = label,                    
                    training_frame = train,                    
                    validation_frame = validation)

xgboost Model Build progress: |

/home/sysadm/.local/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1710833982038_1


Model Summary: 
    number_of_trees
--  -----------------
    25

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.056320671430990975
RMSE: 0.23731976620372558
MAE: 0.15387330072103123
RMSLE: 0.1625879600019455
Mean Residual Deviance: 0.056320671430990975

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.1340022621419143
RMSE: 0.36606319419181477
MAE: 0.24159789309995913
RMSLE: 0.2600635208860518
Mean Residual Deviance: 0.1340022621419143

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 0.12920716116074454
RMSE: 0.3594539764152631
MAE: 0.24030624866066697
RMSLE: 0.25485521532924243
Mean Residual Deviance: 0.12920716116074454

Cross-Validation Metrics Summary: 
                        mean       sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ---------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
aic                     nan        0           nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
loglikelihood           nan        0           nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
mae                     0.240275   0.00846606  0.231162      0.238525      0.227149      0.244511      0.235185      0.232662      0.245475      0.250745      0.246761      0.250577
mean_residual_deviance  0.129123   0.00887689  0.124545      0.132348      0.110734      0.133312      0.123943      0.125172      0.127125      0.136282      0.134776      0.142997
mse                     0.129123   0.00887689  0.124545      0.132348      0.110734      0.133312      0.123943      0.125172      0.127125      0.136282      0.134776      0.142997
r2                      0.0135829  0.061723    0.0102637     -0.013418     0.00203305    -0.0462926    0.116754      0.0790383     -0.0691171    0.0910768     -0.0260224    -0.0084875
residual_deviance       0.129123   0.00887689  0.124545      0.132348      0.110734      0.133312      0.123943      0.125172      0.127125      0.136282      0.134776      0.142997
rmse                    0.359142   0.0124844   0.35291       0.363796      0.332767      0.36512       0.352056      0.353796      0.356545      0.369163      0.367119      0.37815
rmsle                   0.254673   0.00756562  0.250337      0.257536      0.240239      0.259952      0.24839       0.248184      0.257004      0.259228      0.260941      0.264915

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse        training_mae         training_deviance     validation_rmse      validation_mae       validation_deviance
---  -------------------  ----------  -----------------  -------------------  -------------------  --------------------  -------------------  -------------------  ---------------------
     2024-03-19 13:29:32  12.965 sec  0.0                0.5                  0.5                  0.25                  0.5                  0.5                  0.25
     2024-03-19 13:29:32  13.151 sec  1.0                0.4203999070565968   0.4139266205897038   0.17673608185319523   0.43073398438387     0.42252744546418275  0.18553176530320398
     2024-03-19 13:29:32  13.260 sec  2.0                0.3719052622965678   0.3521008671947042   0.1383135241238789    0.3930238551600952   0.36819831526363467  0.15446775072490349
     2024-03-19 13:29:32  13.328 sec  3.0                0.34118367201128524  0.30660032994670955  0.11640629804710426   0.3742421210028614   0.3303210632634891   0.14005716513272035
     2024-03-19 13:29:32  13.584 sec  4.

In [16]:
#export model usingone line of code
xgboost_model.download_mojo(path="~/Documents/sem_2_Biswajit_rana/RKMVERI_2ND/class/ETBDC/h2O/output", get_genmodel_jar=True)

'/home/sysadm/Documents/sem_2_Biswajit_rana/RKMVERI_2ND/class/ETBDC/h2O/output/XGBoost_model_python_1710833982038_1.zip'